In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
from langchain_core.messages import SystemMessage, HumanMessage

model = ChatGroq(model="llama-3.3-70b-versatile", groq_api_key=groq_api_key)
output = model.invoke(
    [
        SystemMessage(
            content="Your help people with their resume and portfolio building "
        ),
        HumanMessage(
            content="Hey I am Amar, and I am an aspiring GenAI engineer"
        ),
    ]   
)

output.content

"Nice to meet you, Amar. As an aspiring GenAI engineer, you're likely excited about the vast potential of generative AI and its applications. Building a strong resume and portfolio can help you stand out in this competitive field. \n\nTo get started, can you tell me a bit more about your background and experience? What relevant skills or projects do you have under your belt? This will help me provide more tailored guidance on how to showcase your strengths and highlight your potential as a GenAI engineer.\n\nAlso, what are your goals? Are you looking to land a job at a top tech company, or perhaps pursue a research position in academia? Knowing your objectives will help me provide more focused advice on how to structure your resume and portfolio."

In [3]:
from langchain_core.messages import AIMessage

output = model.invoke(
    [
        HumanMessage(
            content="Hey I am Amar, and I am an aspiring GenAI engineer"
        ),
        AIMessage(
            content="That's great to hear, Amar! As an aspiring GenAI engineer, what specific areas are you looking to focus on in your resume and portfolio? Are there any particular skills or projects you'd like to highlight?"
        ),
        HumanMessage(
            content="Do you remember my name"
        ),
    ]   
)

output.content

"Your name is Amar. I'm glad we met earlier. How can I assist you today with your GenAI engineering journey?"

In [4]:
!pip install langchain_community

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)-> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model, get_session_history)

In [6]:
config={"configurable":{"session_id":"session_id"}}

with_message_history.invoke(
    [HumanMessage(
        content="Hey I am Amar, and I am an aspiring GenAI engineer"
    ),],
    config=config,
)

AIMessage(content="Nice to meet you, Amar.  As an aspiring GenAI engineer, you're likely interested in developing and working with highly advanced AI systems that can generate human-like text, images, and other forms of media. \n\nGenAI engineering is a rapidly evolving field, with new breakthroughs and innovations emerging regularly. What specific areas of GenAI are you most interested in or would like to explore further? Are you looking to work on natural language processing, computer vision, or perhaps multimodal models? \n\nAlso, what's your current level of experience with AI and programming? Are you looking for resources, guidance, or just want to discuss the latest developments in the field?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 135, 'prompt_tokens': 48, 'total_tokens': 183, 'completion_time': 0.490909091, 'prompt_time': 0.002764207, 'queue_time': 0.059328263, 'total_time': 0.493673298}, 'model_name': 'llama-3.3-70b-versatile', 'system_f

In [7]:
with_message_history.invoke(
    [HumanMessage(
        content="Do you remember my name"
    ),],
    config=config,
)

AIMessage(content="Your name is Amar. I'm glad we met earlier as an aspiring GenAI engineer. How can I assist you today, Amar?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 197, 'total_tokens': 225, 'completion_time': 0.113595162, 'prompt_time': 0.01488773, 'queue_time': 0.05922881299999999, 'total_time': 0.128482892}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_72a5dc99ee', 'finish_reason': 'stop', 'logprobs': None}, id='run-d8080c0f-f0f3-447b-bad4-1423ddc33914-0', usage_metadata={'input_tokens': 197, 'output_tokens': 28, 'total_tokens': 225})

In [14]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You help answer questions with the best of your ability in {language}"),
        MessagesPlaceholder(variable_name="messages")      
    ]
)

chain=prompt|model

In [15]:
response=chain.invoke({"messages":[HumanMessage(content="Hey I am Amar, and I am an aspiring GenAI engineer")],"language":"Hindi"})
response.content

'नमस्ते अमर, मैं आपकी मदद करने में खुश हूँ। जेनेरल आर्टिफिशियल इंटेलिजेंस (GenAI) इंजीनियर बनने का आपका सपना बहुत ही रोमांचक है। यह क्षेत्र तेजी से विकसित हो रहा है और इसमें बहुत सारे अवसर हैं।\n\nआपको GenAI इंजीनियर बनने के लिए क्या कदम उठाने होंगे? क्या आपके पास कोई विशिष्ट योजना या लक्ष्य है? मैं आपको इस यात्रा में मदद करने के लिए तैयार हूँ।'

In [19]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
    )
    

In [ ]:
config={"configurable":{"session_id":"chat_id"}}

response=with_message_history.invoke(
    {"messages":[HumanMessage(content="Hey I am Amar")],"language":"Hindi"},
    config=config
)
response.content

'नमस्ते अमर! आपका स्वागत है! मैं आपकी मदद करने के लिए तैयार हूँ। क्या हाल है? कुछ पूछना चाहते हैं या बस बात करना चाहते हैं?'

In [36]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer=trim_messages(
    max_tokens=100,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=True,
    start_on="human"
    )

messages = [
    SystemMessage(content="You help answer questions with the best of your ability in Hindi"),
    HumanMessage(content="Hey I am Amar, and I am an aspiring GenAI engineer"),
    AIMessage(content="Hello Amar! It's great to meet you. How can I assist you today?"),
    HumanMessage(content="Do you remember my name"),
    AIMessage(content="Yes, your name is Amar. How can I help you today?"),
    HumanMessage(content="I like to play cricket"),
    AIMessage(content="I'm glad to hear that! Cricket is a great sport. Do you play it often?"),
    HumanMessage(content="Yes, I play every weekend"),
    AIMessage(content="That's awesome! Playing every weekend sounds like a lot of fun. Do you play in a team or just with friends?")
]

trimmer.invoke(messages)
    

[SystemMessage(content='You help answer questions with the best of your ability in Hindi', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Do you remember my name', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Yes, your name is Amar. How can I help you today?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like to play cricket', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'm glad to hear that! Cricket is a great sport. Do you play it often?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Yes, I play every weekend', additional_kwargs={}, response_metadata={}),
 AIMessage(content="That's awesome! Playing every weekend sounds like a lot of fun. Do you play in a team or just with friends?", additional_kwargs={}, response_metadata={})]

In [41]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
)

chain.invoke(
    {"messages":messages + [HumanMessage(content="When sport do I play ?")],
     "language":"English"}
).content
    

'आप क्रिकेट खेलते हैं । (You play cricket.)'

In [42]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
    )

config={"configurable":{"session_id":"chat_id2"}}

In [43]:
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="Who am I ?")],"language":"English"},
    config=config
)
response.content

"Unfortunately, I don't have enough information to determine who you are. You could be anyone, and I don't have any context or clues to help me guess.\n\nIf you'd like to play a game or have some fun, you could give me some hints or clues about yourself, and I can try to guess who you are. Alternatively, you could ask me a question or start a conversation, and I can respond accordingly.\n\nSo, who are you? Want to give me a hint?"